In [1]:
import utils
from pathlib import Path
import pandas as pd

In [2]:
df = utils.load_transactions(Path("../data/jointexpenses/"))
df = utils.clean(df)
print(df.shape)
df = utils.deduplicate_transactions(df)
print(df.shape)
df = utils.classify_transactions(df)

other_transactions = df[df.classification == "Other"]
ground_truth_transactions = df[df.classification != "Other"]

X = ground_truth_transactions.desc
y = ground_truth_transactions.classification

(1706, 5)
Found 507 duplicated entries 
and 0 duplicates on the same day from the same file.
(765, 5)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
# print(vect.vocabulary_)

In [5]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [6]:
config = {"class_weight":"balanced"}
model = LinearSVC(**config)
model.fit(X_train_dtm, y_train)
y_pred_class = model.predict(X_test_dtm)
print("Accuracy: ", accuracy_score(y_test, y_pred_class))

Accuracy:  0.9836065573770492
